# IN4080: obligatory assignment 1 (Autumn 2025)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Friday, September 12 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

First, make sure that all required modules are installed:

In [1]:
%pip install tqdm numpy scikit_learn

Note: you may need to restart the kernel to use updated packages.


## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [2]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""
    
    pattern = r'\s*([\.,:;!?\-\(\)\"]+)\s*|\s+'
    result = [token for token in re.split(pattern, text) if token]

    return result

test_text = "Pierre, who works at NR, also teaches at UiO."
print(basic_tokenize(test_text))
print(f"Tokens: {len(basic_tokenize(test_text))}")

['Pierre', ',', 'who', 'works', 'at', 'NR', ',', 'also', 'teaches', 'at', 'UiO', '.']
Tokens: 12


We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [3]:
with open('ndt_test_lm.txt') as f:
    ndt_test_lm_content = f.read()

tokens = basic_tokenize(ndt_test_lm_content)

print(len(tokens))
print(len(set(tokens)))

258330
30103


Number of tokens: 258 330

Types: 30 103

---

We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [4]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

In [ ]:
tokenizer = BPETokenizer('ndt_train_lm.txt')
tokens = tokenizer(ndt_test_lm_content)
tokens = list(tokens)

print(f"Number of tokens: {len(tokens)}")
print(f"Types: {len(set(tokens))}")

Counting word occurrences in corpus ndt_train_lm.txt...

Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226096 [00:00<?, ?it/s]

386099
4408


Comparison: There are significantly more tokens with the BPE tokenizer compared to the basic tokenizer (386 099 vs. 258 330), and significantly fewer types (4408 vs. 30 103). This is because BPE breaks rare words down into more common subwords, increasing token count and reducing vocab size.

---

__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

In [ ]:
import string

def new_get_most_common_pairs(
    self, vocabulary_counts: Dict[str, int], n: int = 200
) -> List[Tuple[str, str]]:
        
    punctuation_chars = set(string.punctuation)

    pair_freqs = {}
    for word, word_count in vocabulary_counts.items():
        subwords = self.tokenize_word(word)
        for i in range(len(subwords) - 1):
            s1, s2 = subwords[i], subwords[i + 1]

            s1_is_alpha = s1.isalpha()
            s2_is_alpha = s2.isalpha()

            # if non-empty and all chars are punctuation
            s1_is_punct = len(s1) > 0 and all(c in punctuation_chars for c in s1)
            s2_is_punct = len(s2) > 0 and all(c in punctuation_chars for c in s2)

            # skip if one is alphanumeric and the other is punctuation
            if (s1_is_alpha and s2_is_punct) or (s1_is_punct and s2_is_alpha):
                continue

            byte_pair = (s1, s2)
            pair_freqs[byte_pair] = (
                pair_freqs.get(byte_pair, 0) + word_count
            )

    most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
    return most_freq_pairs

setattr(BPETokenizer, "get_most_common_pairs", new_get_most_common_pairs)

__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published last year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

- Why can't LLMs do super simple string processing tasks like reversing a string? Tokenization

Explanation: LLMs don't "see" or output individual letters, but tokens. A string such as "reversing" may be tokenized as ["re", "vers", "ing"], meaning the model doesn't even see the raw characters in order to manipulate them.

- Why is LLM bad at simple arithmetic? Tokenization

Explanation: LLMs see numbers as text, not actual numbers, and predicts text based on patterns, not performing actual math.

- Why is LLM worse at non-English languages (e.g. Japanese)? Tokenization

Explanation: Because Japanese doesn't have spaces, and a meaningful word may consist of several characters which are split.

- Why is LLM not actually end-to-end language modeling? Tokenization

Explanation: They don't operate on raw text but tokens created by a pre-defined tokenizer. This breaks the connection between input and the model.

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [7]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()
 

    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [11]:
from collections import defaultdict, Counter

def __init__(
    self,
    training_corpus_file: str,
    tokenizer: "BPETokenizer",
    ngram_size: int = 3,
    alpha_smoothing: float = 1,
):
    LanguageModel.__init__(self, tokenizer)
    self.ngram_size = ngram_size
    self.ngram_probs = {}

    self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

    with open(training_corpus_file, "r", encoding="utf-8") as f:
        text = f.read()

    tokens = list(self.tokenizer(text))

    ngram_counts = defaultdict(int)
    context_counts = defaultdict(int)

    for i in range(len(tokens) - ngram_size + 1):
        ngram = tuple(tokens[i : i + ngram_size])
        context = ngram[:-1]
        ngram_counts[ngram] += 1
        context_counts[context] += 1

    vocab_size = len(self.tokenizer.vocab)

    for context, count_context in context_counts.items():
        denominator = count_context + alpha_smoothing * vocab_size
        default_prob = alpha_smoothing / denominator
        self.ngram_probs[context] = defaultdict(lambda p=default_prob: p)

    for ngram, count_ngram in ngram_counts.items():
        context = ngram[:-1]
        token = ngram[-1]
        count_context = context_counts[context]
        denominator = count_context + alpha_smoothing * vocab_size
        numerator = count_ngram + alpha_smoothing
        self.ngram_probs[context][token] = numerator / denominator

setattr(NGramLanguageModel, '__init__', __init__)

First, the code goes through the training text to count how often n-grams and their contexts appear. We can't use these counts for probs, because any sequence not seen would get a prob of zero. Laplace smoothing fixes this by addin a small value to every count.

__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [13]:
model = NGramLanguageModel("ndt_train_lm.txt", tokenizer, 3, 1)

  0%|          | 0/872824 [00:00<?, ?it/s]

In [16]:
with open('ndt_test_lm.txt') as f:
    test_content = f.read()

print(model.get_perplexity(test_content))

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/386099 [00:00<?, ?it/s]

1708.7573547336558


__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [31]:
def prepare_data(file_path, tokenizer, word_to_idx=None):
    print(f"prepare_data() file: {file_path}")
    
    labels = []
    tokenized_sentences = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if '\t' in line:
                sentence, label = line.rsplit('\t', 1)
                tokens = list(tokenizer(sentence, show_progress_bar=False))
                tokenized_sentences.append(tokens)
                labels.append(label)

    if word_to_idx is None:
        all_tokens = [token for sent in tokenized_sentences for token in sent]
        vocab = sorted(list(set(all_tokens)))
        word_to_idx = {word: i for i, word in enumerate(vocab)}

    N = len(tokenized_sentences)
    V = len(word_to_idx)
    print(f"Samples: {N}. Vocab size: {V}.")
    X = np.zeros((N, V), dtype=np.int8)

    for i, tokens in enumerate(tokenized_sentences):
        for token in tokens:
            # ignore tokens not in training vocab
            if token in word_to_idx:
                j = word_to_idx[token]
                X[i, j] = 1
    
    y = np.array([1 if label == 'nno' else 0 for label in labels], dtype=np.int8)

    return X, y, word_to_idx

In [32]:
X_train, y_train, word_to_idx = prepare_data('ndt_train_class.txt', tokenizer)

prepare_data() file: ndt_train_class.txt
Samples: 29905. Vocab size: 4880.


__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", random_state=42)

print("\nTraining the Logistic Regression model...")
model.fit(X_train, y_train)
print("Model training complete.")


Training the Logistic Regression model...
Model training complete.


__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [34]:
from sklearn.metrics import accuracy_score, classification_report

X_test, y_test, _ = prepare_data('ndt_test_class.txt', tokenizer, word_to_idx)

y_pred_test = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_test)
print(f"\nAccuracy on test data: {accuracy:.4f}\n")

target_names = ["Bokmål", "Nynorsk"]
report = classification_report(y_test, y_pred_test, target_names=target_names)
print(report)

prepare_data() file: ndt_test_class.txt
Samples: 7715. Vocab size: 4880.

Accuracy on test data: 0.9421

              precision    recall  f1-score   support

      Bokmål       0.93      0.96      0.95      4086
     Nynorsk       0.95      0.93      0.94      3629

    accuracy                           0.94      7715
   macro avg       0.94      0.94      0.94      7715
weighted avg       0.94      0.94      0.94      7715



__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [37]:
coefficients = model.coef_[0]
idx_to_word = {i: word for word, i in word_to_idx.items()}
sorted_indices = np.argsort(coefficients)

print("Bokmål:")
top_bokmal_indices = sorted_indices[:5]
for i in top_bokmal_indices:
    word = idx_to_word[i]
    coeff_value = coefficients[i]
    print(f"Word: '{word}'".ljust(20) + f"{coeff_value:.4f}")

print("\nNynorsk:")
# last 5 indeces from sorted array, reversed
top_nynorsk_indices = sorted_indices[-5:][::-1]
for i in top_nynorsk_indices:
    word = idx_to_word[i]
    coeff_value = coefficients[i]
    print(f"Word: '{word}'".ljust(20) + f"{coeff_value:.4f}")

Bokmål:
Word: 'sier '       -3.2598
Word: 'ikke '       -3.2201
Word: 'jeg '        -3.0695
Word: 'fra '        -2.9069
Word: 'Jeg '        -2.7212

Nynorsk:
Word: 'ikkje '      4.2222
Word: 'frå '        4.0436
Word: 'eit '        3.9548
Word: 'dei '        3.5282
Word: 'ein '        3.4037


The weights make sense. All the words are mutually exclusive to their respective form. Ikke/ikkje, jeg/eg, etc.